In [ ]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 


import bk.load
import bk.signal
import bk.plot


import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [ ]:
# bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
# bk.load.session = 'Rat08-20130713'


# session = 'Rat08-20130713'
# neurons,shanks = bk.load.loadSpikeData(bk.load.path)

# run = scipy.io.loadmat(bk.load.path + 'runintervals.mat')
# run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

In [158]:
def computeRateMap(neuron,pos,nxbins=10,nybins=10):
    
    """
        neuron : as Tsd from neuroseries
        pos : as Tsd from neuroseries
        
        output : 
        rate_map : Ratemap in Hz 
        spike_count : number of spikes per spatial bin
        occupancy : number of pos in spatial bin
    """
    
    
    dt = scipy.stats.mode(np.diff(pos.as_units('s').index))[0][0]
    
    pos_spike = pos.realign(neuron)
    
    xbins = np.linspace(pos.x.min(),pos.x.max()+0.01,nxbins)
    ybins = np.linspace(pos.y.min(),pos.y.max()+0.01,nybins)
    
    spike_count,_,_ = np.histogram2d(pos_spike.y,pos_spike.x,[ybins,xbins])
    occupancy,_,_ = np.histogram2d(pos.y,pos.x,[ybins,xbins])
    
    rate_map = spike_count*dt/(occupancy+1)
    rate_map = pd.DataFrame(index = ybins[0:-1]+np.diff(ybins)/2,columns=xbins[0:-1]+np.diff(xbins)/2, data = rate_map)
    
    plt.figure()
    plt.subplot(211)

    plt.plot(pos['x'], pos['y'], alpha = 0.5)
    plt.scatter(pos_spike['x'], pos_spike['y'], color = 'red', alpha = 0.5, edgecolor = None)
    for i in xbins: plt.axvline(i)
    for i in ybins: plt.axhline(i)
    plt.xlim(xbins[0], xbins[-1])
    plt.ylim(ybins[0], ybins[-1])
    plt.subplot(212)
    plt.imshow(rate_map, origin = 'lower', aspect= 'auto')
    plt.clim([0,0.1])
    
    
    return rate_map,spike_count,occupancy

def spatialInformationScore(spike_count,occupancy):
    
    """
        Initial code by KevinAllen
        Adapted by BK 2/11/20
        
        rate_map : number of spike per spatial bin
        occupancy : number of position per spatial bin
    """
    
    
    spike_count = spike_count[occupancy>0]
    occupancy = occupancy[occupancy>0]
    
    rate_map = spike_count/occupancy
    
    mRate = spike_count.sum()/occupancy.sum()
    prob = occupancy/occupancy.sum()
    
    index = rate_map>0
    
    info = np.sum(prob[index] * (rate_map[index]/mRate) * np.log2(rate_map[index]/mRate))
    return info


In [160]:
path = bk.load.sessions()['Path'][4]
bk.load.current_session(path)

neurons,metadata = bk.load.loadSpikeData(bk.load.path)
pos = bk.load.pos()
states = bk.load.states()

run = scipy.io.loadmat('runintervals.mat')
run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

trackrun = scipy.io.loadmat(bk.load.session+'-TrackRunTimes.mat')
trackrun = nts.IntervalSet(trackrun['trackruntimes'][:,0],trackrun['trackruntimes'][:,1],time_units='s')

pos_run = pos.restrict(run)#.restrict(trackrun)

Rat : 8 on day : 5
Working with session Rat08-20130712 @ Z:\Rat08\Rat08-20130712
Data already saved in Numpy format, loading them from here:
Rat08-20130712-neurons.npy
Rat08-20130712-metadata.npy


In [161]:
theta = scipy.io.loadmat('Billel/ThetaMod/ThetaMod.mat')

In [207]:
info = []
for n,s,i in zip(neurons,metadata['Shank'],metadata['Id']):
    if s == 4:
        _,spike_count,occupancy = computeRateMap(n,pos_run,25,15)
        info.append(spatialInformationScore(spike_count,occupancy))
        plt.suptitle(str(s) + str(i))
MRL = theta['stats']['hpc'][0][0]['r'][0][0].squeeze()

In [154]:
for s,i in zip(metadata['Shank'],metadata['Id']):
    index = np.all([s,i] == theta['metadata'][:,2:4],1)
    if not np.any(index):
        print(s,i)

4 6


In [72]:
info = np.array(info).squeeze()
MRL = np.array(MRL)
data = {'MRL':np.log(MRL),'info':info,'region':metadata['Region'],'type':metadata['Type']}
data = pd.DataFrame(data)
sns.scatterplot('info','MRL',data = data,hue = 'region',style = 'type')

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='info', ylabel='MRL'>